# Facefusion WebUI
## 阿里云DSW 一键脚本 By bilibili@十字鱼
### 参考感谢——sukaka
- 十字鱼 https://space.bilibili.com/893892

## 1.安装并运行Facefusion
- 如果启动不成功，请停止后重复运行第2个代码框。

In [12]:
# 学术资源加速
import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

In [ ]:
# 这里是第2个代码框，用于下载模型和依赖，但是下不全，有一些需要手动下载
# 且第一次执行过以后就不用执行这里，跑的太慢了，直接用快速启动
!pip install onnxruntime
import os
import time
from concurrent.futures import ProcessPoolExecutor,ThreadPoolExecutor
#安装目录
path = "/root/autodl-tmp"
#安装网址
url = "https://github.moeyy.xyz/https://github.com/Wild-Piggggggg/facefusion"
repo = url.split('/')[-1]
#是否重装
reinstall = False
#修改分支
branch = "master"
#模型文件（先换行，再加链接）  （注意这里的大小写，模型名称要和最终的模型路径统一，如果报错模型不存在大概率是这里出了问题）
model_urls = """
https://github.com/facefusion/facefusion-assets/releases/download/models/inswapper_128.onnx
https://github.com/facefusion/facefusion-assets/releases/download/models/inswapper_128_fp16.onnx
https://github.com/facefusion/facefusion-assets/releases/download/models/GFPGANv1.4.onnx
https://github.com/facefusion/facefusion-assets/releases/download/models/GFPGANv1.3.onnx
https://github.com/facefusion/facefusion-assets/releases/download/models/GFPGANv1.2.onnx
https://github.com/facefusion/facefusion-assets/releases/download/models/codeformer.onnx
https://github.com/facefusion/facefusion-assets/releases/download/models/GPEN-BFR-512.onnx
https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.1/RealESRGAN_x2plus.pth
https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth
https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.1/RealESRNet_x4plus.pth
https://github.com/facefusion/facefusion-assets/releases/download/models/arcface_simswap.onnx
https://github.com/facefusion/facefusion-assets/releases/download/models/arcface_simswap.onnx
https://github.com/facefusion/facefusion-assets/releases/download/models/arcface_w600k_r50.onnx
https://github.com/facefusion/facefusion-assets/releases/download/models/blendswap_256.onnx
https://github.com/facefusion/facefusion-assets/releases/download/models/face_occluder.onnx
https://github.com/facefusion/facefusion-assets/releases/download/models/gender_age.onnx
https://github.com/facefusion/facefusion-assets/releases/download/models/gpen_bfr_256.onnx
https://github.com/facefusion/facefusion-assets/releases/download/models/gpen_bfr_512.onnx
https://github.com/facefusion/facefusion-assets/releases/download/models/open_nsfw.onnx
https://github.com/facefusion/facefusion-assets/releases/download/models/restoreformer.onnx
https://github.com/facefusion/facefusion-assets/releases/download/models/retinaface_10g.onnx
https://github.com/facefusion/facefusion-assets/releases/download/models/simswap_256.onnx
https://github.com/facefusion/facefusion-assets/releases/download/models/simswap_512_unofficial.onnx
https://github.com/facefusion/facefusion-assets/releases/download/models/yunet_2023mar.onnx
"""
#额外文件（先换行，再加链接）（下载到安装目录，方便剪切）
extra_urls = """
"""
#内存优化（不要修改）
def libtcmalloc():
    print('开始安装内存优化')
    !git config --global http.postBuffer 2000000000
    !apt-get update -qq > /dev/null 2>&1
    !apt-get install -qq -y aria2 > /dev/null 2>&1
    !apt-get install -qq -y zip > /dev/null 2>&1
    !apt-get install -qq -y ffmpeg  > /dev/null 2>&1
    if os.path.exists(f'{path}/temp'):
        os.environ["LD_PRELOAD"] = "libtcmalloc.so"
        print('内存优化已安装')
    else:
        os.chdir(f'{path}')
        os.makedirs('temp', exist_ok=True)
        os.chdir('temp')
        os.system('wget -q http://launchpadlibrarian.net/367274644/libgoogle-perftools-dev_2.5-2.2ubuntu3_amd64.deb')
        os.system('wget -q https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/google-perftools_2.5-2.2ubuntu3_all.deb')
        os.system('wget -q https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/libtcmalloc-minimal4_2.5-2.2ubuntu3_amd64.deb')
        os.system('wget -q https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/libgoogle-perftools4_2.5-2.2ubuntu3_amd64.deb')
        os.system('apt-get install -qq libunwind8-dev -y > /dev/null 2>&1')
        !dpkg -i *.deb > /dev/null 2>&1
        os.environ["LD_PRELOAD"] = "libtcmalloc.so"
        !rm *.deb > /dev/null 2>&1
        print('内存优化安装完毕')
#安装主体（不要修改）
def install_main():
    print(f'开始安装{repo}')
    !git -C {path} clone {url} > /dev/null 2>&1
    !git -C {path}/{repo} checkout {branch}
    !git -C {path}/{repo} pull
    if os.path.exists(f'{path}/{repo}'):
        %cd {path}/{repo}
        print('开始安装依赖，请耐心等待')
        !pip install -U pip > /dev/null 2>&1
        !pip install -U -r {path}/{repo}/requirements.txt > /dev/null 2>&1
        !pip uninstall -y onnxruntime onnxruntime-gpu > /dev/null 2>&1
        !pip install onnxruntime-gpu==1.16.0 > /dev/null 2>&1
        !pip install -U torch==2.0.1 torchvision torchaudio > /dev/null 2>&1
        !pip install -U ipywidgets > /dev/null 2>&1
        %cd {path}
        print(f'{repo}安装完毕')
#下载文件（不要修改）
def download_all(times):
    tasks = []
    for i in range(times):
        tasks.extend(download_aria(model_urls,f'{path}/{repo}/.assets/models'))
        tasks.extend(download_aria('https://github.com/deepinsight/insightface/releases/download/v0.7/buffalo_l.zip','/root/.insightface/models'))
        tasks.extend(download_aria(extra_urls,f'{path}'))
    with ThreadPoolExecutor() as executor:
        for task in tasks:
            executor.submit(os.system, task)
#下载模块（不要修改）
def download_aria(links,folder):
    global check
    tasks = []
    link = links.strip().split("\n")
    for li in link:
        if "https://github.moeyy.xyz/https://github.com" in li:
            li=li.replace("https://github.moeyy.xyz/https://github.com","https://github.moeyy.xyz/https://github.com")
        elif "https://github.com" in li:
            li=li.replace("https://github.com","https://github.moeyy.xyz/https://github.com")
        elif "huggingface.co" in li:
            li=li.replace("huggingface.co","huggingface.sukaka.top")
        elif "civitai.com" in li:
            li=li.replace("civitai.com","civitai.sukaka.top")
        fi = li.split('/')[-1]
        if "." in fi:
            tasks.append(f'aria2c --console-log-level=error -c -x 16 -s 16 -k 1M -t 10 -d {folder} -o {fi} "{li}" > /dev/null 2>&1')
            check = check & os.path.exists(f'{folder}/{fi}')
        else:
            tasks.append(f'aria2c --console-log-level=error -c -x 16 -s 16 -k 1M -t 10 -d {folder} "{li}" > /dev/null 2>&1')
    return tasks
#主进程（不要修改）
def main():
    time_start = time.time()
    global check
    print("运行开始")
    !df -hl #查看磁盘
    !nvidia-smi #查看显卡
    %cd {path}
    if reinstall:
        print('旧文件删除中')
        !rm -rf {path}/{repo}
    with ProcessPoolExecutor() as executor:
        futures = []
        for func in [libtcmalloc,install_main]:
            futures.append(executor.submit(func))
            time.sleep(1)
        for future in futures:
            future.result()
    if os.path.exists(f'{path}/{repo}'):
        print("开始下载，请耐心等待")
        n = 1
        while not check:
            print("第",n,"次下载")
            n = n+1
            check = True
            download_all(1)
            if not check:
                download_all(1)
        time_end = time.time()
        !unzip -n /root/.insightface/models/buffalo_l.zip -d /root/.insightface/models/buffalo_l
        print('\033[32m安装耗时:',int((time_end - time_start)/60),'min\033[0m')
        %cd {path}/{repo}
        !python run.py -o {path}/temp --skip-download --execution-providers cuda --execution-thread-count 16
    else:
        print('安装失败请重试')

check = False
if __name__ == '__main__':
    main()

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 49.5 MB/s eta 0:00:00a 0:00:01
运行开始
Filesystem                         Size  Used Avail Use% Mounted on
overlay                             30G   20G   11G  66% /
tmpfs                               64M     0   64M   0% /dev
tmpfs                              504G     0  504G   0% /sys/fs/cgroup
shm                                 40G     0   40G   0% /dev/shm
/dev/md0                           150M     0  150M   0% /init
tmpfs                              504G   12K  504G   1% /proc/driver/nvidia
tmpfs                              504G  4.0K  504G   1% /etc/nvidia/nvidia-application-profiles-rc.d
/dev/mapper/ubuntu--vg-ubuntu--lv  438G   18G  398G   5% /usr/bin/nvidia-smi
tmpfs                              504G     0  504G   0% /proc/asound
tmpfs                              504G     0  504G   0% /proc/acpi
tmpfs                              504G     0  504G   0% /proc

## 2.快速启动

In [6]:
%cd /root/autodl-tmp/facefusion
!python run.py -o /root/autodl-tmp/temp --skip-download --execution-providers cuda --execution-thread-count 16

/root/autodl-tmp/facefusion
Running on local URL:  http://127.0.0.1:6006

To create a public link, set `share=True` in `launch()`.
Traceback (most recent call last):
  File "/root/miniconda3/lib/python3.10/site-packages/gradio/routes.py", line 534, in predict
    output = await route_utils.call_process_api(
  File "/root/miniconda3/lib/python3.10/site-packages/gradio/route_utils.py", line 226, in call_process_api
    output = await app.get_blocks().process_api(
  File "/root/miniconda3/lib/python3.10/site-packages/gradio/blocks.py", line 1550, in process_api
    result = await self.call_function(
  File "/root/miniconda3/lib/python3.10/site-packages/gradio/blocks.py", line 1185, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "/root/miniconda3/lib/python3.10/site-packages/anyio/to_thread.py", line 33, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "/root/miniconda3/lib/python3.10/site-packages/anyio/_backends/_asyncio.py", line 87